Import libraries  *DO NOT EDIT

In [1]:
import PyPDF2                    # open and pdf manipulation
import os                        # change working directory
import pandas as pd              # df manipulation
from datetime import date        # extract date for csv title output

ModuleNotFoundError: No module named 'PyPDF2'

Inputs (user specific)

In [2]:
# directory locations of documents
papers_dir = 'C:\\Users\\redinger\\Documents\\GLANSIS_folder\\Endnote_species\\Pistia stratiotes\\Pistia stratiotes PDFs'
csv_dir = 'C:\\Users\\redinger\\Documents\\GLANSIS_folder\\Endnote_species\\Pistia stratiotes\\P.stratiotes_references_5.21.csv'
export_dir = 'C:\\Users\\redinger\\Documents\\GLANSIS_folder\\Endnote_species\\Pistia stratiotes'

# enter scientific and common names so they can be added to keywords -- keep names between quotation marks and be sure to use commas
names = 'Pistia stratiotes, water lettuce' #used for keywords
# scientific_name = 'Pistia_stratiotes' #used only for naming documents




Extract Keywords *DO NOT EDIT

In [3]:
# Function to extract keywords from metadata in text
def extract_keywords_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PyPDF2.PdfReader(file)
        try:
            pdf_keywords = pdf.metadata['/Keywords'].replace(';',',')
        except KeyError:
            pdf_keywords = ''
    return pdf_keywords
## Note to self - this does not capture everything. Need to come with better alternative to except KeyError:


# Pull keywords from pdfs in folder file
results = []
for filename in os.listdir(papers_dir):
    if filename.endswith(".pdf"):
        file_path = os.path.join(papers_dir, filename)
        pdf_keywords = extract_keywords_from_pdf(file_path)
        results.append([filename, pdf_keywords])
        

# convert to data frame
col_names = ["file.name", "pdf_keywords"]
df = pd.DataFrame(results, columns = col_names)


# combine keywords with common and scientific names
df['keywords'] = names + ', ' + df['pdf_keywords'].astype(str)


Clean keywords - remove duplicates and extra characters from keywords *DO NOT EDIT

In [4]:
# drop original pdf_keywords
df.drop('pdf_keywords', axis = 1, inplace = True)


# function to remove duplicates from addition of common and scientific names
def remove_duplicates(keywords_col):
    words = keywords_col.split(',')
    unique_words = list(set(words))
    return ', '.join(unique_words)

df['keywords'] = df['keywords'].apply(remove_duplicates)


# function to remove anything before first keyword
def remove_pre(keywords_col):
    first_letter_index = next((index for index, char in enumerate(keywords_col) if char.isalpha()), -1)
    if first_letter_index != -1:
        return keywords_col[first_letter_index:]
    return keywords_col

df['keywords'] = df['keywords'].apply(remove_pre)


# export output to csv file
os.chdir(export_dir)
df.to_csv('keywords' + '_' + scientific_name + '_' + str(date.today()) + '.csv', index = False)


Combine with csv file

In [19]:
# upload csv with clean references
ref = pd.read_csv(csv_dir, encoding = 'cp1252')

In [29]:
# merge on file names
output = ref.merge(df, left_on = 'File Attachments', right_on = 'file.name', how = 'left')

In [32]:
# export new file
output.to_csv(scientific_name + '_' + str(date.today()) + '.csv', index = False, encoding = 'cp1252')